**Eric Meinhardt / emeinhardt@ucsd.edu**

In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Reading the CMU dict in

In [2]:
import csv

CMU pronouncing dictionary v0.7b obtained from `http://svn.code.sf.net/p/cmusphinx/code/trunk/cmudict/cmudict-0.7b`

In [3]:
%ls

 cmudict-0.7b                      cmudict-0.7b.tsv
 cmudict-0.7b_IPA_destressed.tsv  'CMU to IPA.ipynb'
 cmudict-0.7b_IPA_stressed.tsv


In [4]:
cmudict_raw_filename = 'cmudict-0.7b'

cmudict_raw = []
with open(cmudict_raw_filename, 'r', encoding='latin-1') as the_file:
    for row in the_file:
        cmudict_raw.append(row.rstrip('\n'))#(row.rstrip('\r\n'))

In [5]:
%cat -n cmudict-0.7b | head -60

     1	;;; # CMUdict  --  Major Version: 0.07
     2	;;; 
     3	;;; # $HeadURL$
     4	;;; # $Date::                                                   $:
     5	;;; # $Id::                                                     $:
     6	;;; # $Rev::                                                    $: 
     7	;;; # $Author::                                                 $:
     8	;;;
     9	;;; #
    10	;;; # ========================================================================
    11	;;; # Copyright (C) 1993-2015 Carnegie Mellon University. All rights reserved.
    12	;;; #
    13	;;; # Redistribution and use in source and binary forms, with or without
    14	;;; # modification, are permitted provided that the following conditions
    15	;;; # are met:
    16	;;; #
    17	;;; # 1. Redistributions of source code must retain the above copyright
    18	;;; #    notice, this list of conditions and the following disclaimer.
    19	;;; #    The contents of this file are deemed to be so

In [6]:
cmudict_raw[0] #header...
cmudict_raw[55] #end header
cmudict_raw[56]

';;; # CMUdict  --  Major Version: 0.07'

';;; '

'!EXCLAMATION-POINT  EH2 K S K L AH0 M EY1 SH AH0 N P OY2 N T'

In [7]:
cmudict_entry_lines = cmudict_raw[56:]
cmudict_entry_lines[0]
cmudict_entry_lines[2434]

'!EXCLAMATION-POINT  EH2 K S K L AH0 M EY1 SH AH0 N P OY2 N T'

'ALBINIA  AA0 L B IY1 N IY0 AH0'

# Convert to tsv

Each entry line is split by a double space...

In [8]:
cmudict_entry_lines[5].split('  ')
cmudict_entry_lines[2434].split('  ')
cmudict_entry_lines[89347].split('  ')

['"IN-QUOTES', 'IH1 N K W OW1 T S']

['ALBINIA', 'AA0 L B IY1 N IY0 AH0']

['PATRICOF', 'P AE1 T R IH0 K AO2 F']

In [9]:
tab_separated_cmudict_lines = [entry.split('  ') for entry in cmudict_entry_lines]
tab_separated_cmudict_lines[:10]

[['!EXCLAMATION-POINT', 'EH2 K S K L AH0 M EY1 SH AH0 N P OY2 N T'],
 ['"CLOSE-QUOTE', 'K L OW1 Z K W OW1 T'],
 ['"DOUBLE-QUOTE', 'D AH1 B AH0 L K W OW1 T'],
 ['"END-OF-QUOTE', 'EH1 N D AH0 V K W OW1 T'],
 ['"END-QUOTE', 'EH1 N D K W OW1 T'],
 ['"IN-QUOTES', 'IH1 N K W OW1 T S'],
 ['"QUOTE', 'K W OW1 T'],
 ['"UNQUOTE', 'AH1 N K W OW1 T'],
 ['#HASH-MARK', 'HH AE1 M AA2 R K'],
 ['#POUND-SIGN', 'P AW1 N D S AY2 N']]

In [10]:
cmudict = list(map(lambda tscdl: {'Orthography':tscdl[0],
                                  'Transcription':tscdl[1]},
                   tab_separated_cmudict_lines))

In [11]:
from csv import DictWriter

In [12]:
cmudict_tsv_fn = 'cmudict-0.7b.tsv'

with open(cmudict_tsv_fn, 'w', newline='\n') as dl:
    fieldnames = ['Orthography', 'Transcription']
    writer = csv.DictWriter(dl, fieldnames=fieldnames, delimiter='\t', quoting=csv.QUOTE_NONE, quotechar='@')
    writer.writeheader()
    for entry in cmudict:
        _ = writer.writerow(entry) #assignment suppresses return value (= number of characters, iirc?)

In [13]:
%cat -n cmudict-0.7b.tsv | head -60





























































cat: write error: Broken pipe


# Convert wordform representations to dotted strings

In [14]:
def modify_dict(the_dict, the_key, the_new_value):
    '''
    Composable and (naively-implemented) non-mutating dictionary update.
    '''
    new_dict = {k:the_dict[k] for k in the_dict}
    new_dict.update({the_key: the_new_value})
    return new_dict

In [15]:
'AH1 N K W OW1 T'.split(' ')
'.'.join('AH1 N K W OW1 T'.split(' '))

['AH1', 'N', 'K', 'W', 'OW1', 'T']

'AH1.N.K.W.OW1.T'

In [16]:
ds_lexicon = list(map(lambda row: modify_dict(row, 
                                              'Transcription', 
                                              '.'.join( row['Transcription'].split(' ') )),
                      cmudict))
ds_lexicon[:5]

[{'Orthography': '!EXCLAMATION-POINT',
  'Transcription': 'EH2.K.S.K.L.AH0.M.EY1.SH.AH0.N.P.OY2.N.T'},
 {'Orthography': '"CLOSE-QUOTE', 'Transcription': 'K.L.OW1.Z.K.W.OW1.T'},
 {'Orthography': '"DOUBLE-QUOTE', 'Transcription': 'D.AH1.B.AH0.L.K.W.OW1.T'},
 {'Orthography': '"END-OF-QUOTE', 'Transcription': 'EH1.N.D.AH0.V.K.W.OW1.T'},
 {'Orthography': '"END-QUOTE', 'Transcription': 'EH1.N.D.K.W.OW1.T'}]

In [17]:
def tupleToDottedString(pair): 
    return '.'.join(pair)

def dottedStringToTuple(s): 
    return tuple(s.split('.'))

t2ds = tupleToDottedString
ds2t = dottedStringToTuple

In [18]:
ds2t('AH1.N.K.W.OW1.T')

('AH1', 'N', 'K', 'W', 'OW1', 'T')

# Convert ARPABET to IPA

In [19]:
from functools import reduce

In [20]:
def union(Ss):
    return reduce(set.union, Ss)

## Identify all ARPABET symbols in use in the data

In [21]:
transcriptions = list(map(lambda entry: entry['Transcription'],
                          ds_lexicon))
transcriptions[:10]

['EH2.K.S.K.L.AH0.M.EY1.SH.AH0.N.P.OY2.N.T',
 'K.L.OW1.Z.K.W.OW1.T',
 'D.AH1.B.AH0.L.K.W.OW1.T',
 'EH1.N.D.AH0.V.K.W.OW1.T',
 'EH1.N.D.K.W.OW1.T',
 'IH1.N.K.W.OW1.T.S',
 'K.W.OW1.T',
 'AH1.N.K.W.OW1.T',
 'HH.AE1.M.AA2.R.K',
 'P.AW1.N.D.S.AY2.N']

In [22]:
ARPABET_in_cmudict = union([set(t.split('.')) for t in transcriptions])
len(ARPABET_in_cmudict)
print(ARPABET_in_cmudict)

69

{'IY1', 'UH0', 'AH2', 'OW2', 'F', 'IY2', 'DH', 'AO2', 'AH0', 'AO0', 'AA2', 'D', 'OW0', 'G', 'AW0', 'ER1', 'AE1', 'UW0', 'AO1', 'SH', 'AY2', 'ZH', 'OY0', 'R', 'EY1', 'IY0', 'AE0', 'AE2', 'AW2', 'EY0', 'B', 'V', 'UW2', 'Z', 'T', 'AY1', 'AW1', 'EY2', 'IH0', 'M', 'AA0', 'UW1', 'OY2', 'P', 'UH2', 'CH', 'N', 'EH0', 'JH', 'W', 'K', 'EH1', 'ER2', 'ER0', 'OW1', 'AH1', 'OY1', 'NG', 'TH', 'L', 'IH2', 'HH', 'Y', 'IH1', 'AY0', 'UH1', 'S', 'EH2', 'AA1'}


In [23]:
'EY2'[:-1]

'EY'

In [24]:
ARPABET_in_cmudict_nostress = set(map(lambda symbol: symbol if symbol[-1] != "0" and symbol[-1] != "1" and symbol[-1] != "2" else symbol[:-1],
                                       ARPABET_in_cmudict))
ARPABET_in_cmudict_nostress

{'AA',
 'AE',
 'AH',
 'AO',
 'AW',
 'AY',
 'B',
 'CH',
 'D',
 'DH',
 'EH',
 'ER',
 'EY',
 'F',
 'G',
 'HH',
 'IH',
 'IY',
 'JH',
 'K',
 'L',
 'M',
 'N',
 'NG',
 'OW',
 'OY',
 'P',
 'R',
 'S',
 'SH',
 'T',
 'TH',
 'UH',
 'UW',
 'V',
 'W',
 'Y',
 'Z',
 'ZH'}

## Map ARPABET symbols to IPA symbols

See https://en.wikipedia.org/wiki/ARPABET.

In [25]:
arpabetUnicodeIPArelation = set([ \
('AO', 'ɔ'),
('AA', 'ɑ'),
('IY', 'i'),
('UW', 'u'),
('EH', 'ɛ'),
('IH', 'ɪ'),
('UH', 'ʊ'),
('AH', 'ʌ'),
('AX', 'ə'),
('AE', 'æ'),
('EY', 'eɪ'),
('AY', 'aɪ'),
('OW', 'oʊ'),
('AW', 'aʊ'),
('OY', 'ɔɪ'),
('ER', 'ɚ'),
('P', 'p'),
('B', 'b'),
('T', 't'),
('D', 'd'),
('K', 'k'),
('G', 'g'),
('CH', 'tʃ'),
('JH', 'dʒ'),
('F', 'f'),
('V', 'v'),
('TH', 'θ'),
('DH', 'ð'),
('S', 's'),
('Z', 'z'),
('SH', 'ʃ'),
('ZH', 'ʒ'),
('HH', 'h'),
('M', 'm'),
('EM', 'm̩'),
('N', 'n'),
('EN', 'n̩'),
('NG', 'ŋ'),
('ENG', 'ŋ̩'),
('L', 'l'),
('EL', 'l̩'),
('R', 'r'),
('DX', 'ɾ'),
('NX', 'ɾ̃'),
('Y', 'j'),
('W', 'w'),
('Q', 'ʔ')
])

In [26]:
ARPABET_in_relation = set(map(lambda r: r[0],
                              arpabetUnicodeIPArelation))
ARPABET_in_relation

{'AA',
 'AE',
 'AH',
 'AO',
 'AW',
 'AX',
 'AY',
 'B',
 'CH',
 'D',
 'DH',
 'DX',
 'EH',
 'EL',
 'EM',
 'EN',
 'ENG',
 'ER',
 'EY',
 'F',
 'G',
 'HH',
 'IH',
 'IY',
 'JH',
 'K',
 'L',
 'M',
 'N',
 'NG',
 'NX',
 'OW',
 'OY',
 'P',
 'Q',
 'R',
 'S',
 'SH',
 'T',
 'TH',
 'UH',
 'UW',
 'V',
 'W',
 'Y',
 'Z',
 'ZH'}

In [27]:
{symbol for symbol in ARPABET_in_cmudict_nostress if symbol not in ARPABET_in_relation}

set()

The codebox above means every ARPABET symbol in the CMU dictionary is in the ARPABET-IPA relation.

In [28]:
def arpabetToUnicodeIPA(arpabetSymbol):
    mapping = dict(arpabetUnicodeIPArelation)
    #print(mapping)
    return mapping[arpabetSymbol]
arpabetToUnicodeIPA('TH')

def invertMapping(mydict): 
    return dict([[val, key] for key,val in mydict.items()])
def unicodeIPAToArpabet(unicodeIPAsymbol):
    return invertMapping( dict(arpabetUnicodeIPArelation) )[unicodeIPAsymbol]
unicodeIPAToArpabet('θ')

'θ'

'TH'

In [29]:
def IPAifyUnstressedRep(unstressedRep):
    arpabetTuple = dottedStringToTuple( unstressedRep ) 
    
    IPAtuple = tuple(list(map(arpabetToUnicodeIPA, arpabetTuple)))
    
    dottedIPAsymbols = tupleToDottedString( IPAtuple )
    return dottedIPAsymbols

def IPAifyStressedRep(stressedRep):
    stressedArpabetTuple = dottedStringToTuple( stressedRep )

    arpabetSymbols = [symb[:-1] if symb[-1] == '0' or symb[-1] == '1' or symb[-1] == '2' else symb[:] for symb in stressedArpabetTuple]
    stresses = [symb[-1] if symb[-1] == '0' or symb[-1] == '1' or symb[-1] == '2' else '' for symb in stressedArpabetTuple]
#     print('arpabetSymbols:')
#     print(arpabetSymbols)
#     print('stresses:')
#     print(stresses)
#     print('together:')
    
    IPAsymbols = list(map(arpabetToUnicodeIPA, arpabetSymbols))
    stressedIPAtuple = tuple(map(lambda s01: s01[0] + s01[1], zip(IPAsymbols,stresses)))
    
    dottedStressedIPAsymbols = tupleToDottedString( stressedIPAtuple )
    return dottedStressedIPAsymbols

In [30]:
IPAifyStressedRep('EH2.K.S.K.L.AH0.M.EY1.SH.AH0.N.P.OY2.N.T')

'ɛ2.k.s.k.l.ʌ0.m.eɪ1.ʃ.ʌ0.n.p.ɔɪ2.n.t'

## Apply mapping to CMU lexicon

In [31]:
ds_lexicon[:5]

[{'Orthography': '!EXCLAMATION-POINT',
  'Transcription': 'EH2.K.S.K.L.AH0.M.EY1.SH.AH0.N.P.OY2.N.T'},
 {'Orthography': '"CLOSE-QUOTE', 'Transcription': 'K.L.OW1.Z.K.W.OW1.T'},
 {'Orthography': '"DOUBLE-QUOTE', 'Transcription': 'D.AH1.B.AH0.L.K.W.OW1.T'},
 {'Orthography': '"END-OF-QUOTE', 'Transcription': 'EH1.N.D.AH0.V.K.W.OW1.T'},
 {'Orthography': '"END-QUOTE', 'Transcription': 'EH1.N.D.K.W.OW1.T'}]

In [32]:
lexicon_IPA_stress = list(map(lambda entry: modify_dict(entry, 'Transcription', IPAifyStressedRep(entry['Transcription'])),
                              ds_lexicon))

In [33]:
lexicon_IPA_stress[:5]

[{'Orthography': '!EXCLAMATION-POINT',
  'Transcription': 'ɛ2.k.s.k.l.ʌ0.m.eɪ1.ʃ.ʌ0.n.p.ɔɪ2.n.t'},
 {'Orthography': '"CLOSE-QUOTE', 'Transcription': 'k.l.oʊ1.z.k.w.oʊ1.t'},
 {'Orthography': '"DOUBLE-QUOTE', 'Transcription': 'd.ʌ1.b.ʌ0.l.k.w.oʊ1.t'},
 {'Orthography': '"END-OF-QUOTE', 'Transcription': 'ɛ1.n.d.ʌ0.v.k.w.oʊ1.t'},
 {'Orthography': '"END-QUOTE', 'Transcription': 'ɛ1.n.d.k.w.oʊ1.t'}]

In [34]:
def destress(symbol):
    return symbol[:-1] if symbol[-1] == '0' or symbol[-1] == '1' or symbol[-1] == '2' else symbol[:]

def removeStress(stressedRep):
    stressedIPATuple = ds2t(stressedRep)
    IPAtuple = tuple(map(destress,
                         stressedIPATuple))
    return t2ds(IPAtuple)

removeStress('ɛ2.k.s.k.l.ʌ0.m.eɪ1.ʃ.ʌ0.n.p.ɔɪ2.n.t')

'ɛ.k.s.k.l.ʌ.m.eɪ.ʃ.ʌ.n.p.ɔɪ.n.t'

In [35]:
lexicon_IPA_nostress = list(map(lambda entry: modify_dict(entry, 'Transcription', removeStress(entry['Transcription'])),
                                lexicon_IPA_stress))
lexicon_IPA_nostress[:5]

[{'Orthography': '!EXCLAMATION-POINT',
  'Transcription': 'ɛ.k.s.k.l.ʌ.m.eɪ.ʃ.ʌ.n.p.ɔɪ.n.t'},
 {'Orthography': '"CLOSE-QUOTE', 'Transcription': 'k.l.oʊ.z.k.w.oʊ.t'},
 {'Orthography': '"DOUBLE-QUOTE', 'Transcription': 'd.ʌ.b.ʌ.l.k.w.oʊ.t'},
 {'Orthography': '"END-OF-QUOTE', 'Transcription': 'ɛ.n.d.ʌ.v.k.w.oʊ.t'},
 {'Orthography': '"END-QUOTE', 'Transcription': 'ɛ.n.d.k.w.oʊ.t'}]

# Write to file

In [36]:
%ls

 cmudict-0.7b                      cmudict-0.7b.tsv
 cmudict-0.7b_IPA_destressed.tsv  'CMU to IPA.ipynb'
 cmudict-0.7b_IPA_stressed.tsv


In [37]:
stressed_fn = 'cmudict-0.7b_IPA_stressed.tsv'
destressed_fn = 'cmudict-0.7b_IPA_destressed.tsv'

In [38]:
with open(stressed_fn, 'w', newline='', encoding='utf-8') as tsvfile:
    writer = csv.DictWriter(tsvfile, delimiter='\t', fieldnames=['Orthography', 'Transcription'], quoting=csv.QUOTE_NONE, quotechar='@')
    writer.writeheader()
    writer.writerows(lexicon_IPA_stress)

In [39]:
with open(destressed_fn, 'w', newline='', encoding='utf-8') as tsvfile:
    writer = csv.DictWriter(tsvfile, delimiter='\t', fieldnames=['Orthography', 'Transcription'], quoting=csv.QUOTE_NONE, quotechar='@')
    writer.writeheader()
    writer.writerows(lexicon_IPA_nostress)

In [40]:
%cat -n cmudict-0.7b_IPA_stressed.tsv | head -60





























































cat: write error: Broken pipe


In [41]:
%cat -n cmudict-0.7b_IPA_destressed.tsv | head -60





























































cat: write error: Broken pipe
